In [36]:
import pandas as pd
import numpy as np
import json
import requests
import time

In [52]:
# Chargement du jeu de données
df = pd.read_csv('01B - dvf_sans_gps_iris.csv')
df = df.drop(columns = {'Unnamed: 0'})
df = df.rename(columns = {'Commune2': 'Commune Alt'})
df.head()
#print(df.columns)

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18,20,22,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,Surface terrain,Code commune INSEE,Ville,Adresse,Commune Alt,Prix m2,Paris,dep_name,region_name,Mer
0,1,02/01/2020,Vente,268800.0,14.0,NaN,RUE,0411,DE L'ARCADE,75008.0,...,NaN,75108,75008 PARIS 08,14 RUE DE L'ARCADE 75008 PARIS,PARIS,11200.000000,1,Paris,Île-de-France,0
1,1,06/01/2020,Vente,878378.0,9.0,NaN,BD,5888,DE LA MADELEINE,75001.0,...,NaN,75101,75001 PARIS 01,9 BD DE LA MADELEINE 75001 PARIS,PARIS,13308.757576,1,Paris,Île-de-France,0
2,1,10/01/2020,Vente,680000.0,10.0,NaN,RUE,1200,DU BOURG L'ABBE,75003.0,...,NaN,75103,75003 PARIS 03,10 RUE DU BOURG L'ABBE 75003 PARIS,PARIS,13333.333333,1,Paris,Île-de-France,0
3,1,08/01/2020,Vente,370000.0,79.0,NaN,RUE,0420,DES ARCHIVES,75003.0,...,NaN,75103,75003 PARIS 03,79 RUE DES ARCHIVES 75003 PARIS,PARIS,17619.047619,1,Paris,Île-de-France,0
4,1,09/01/2020,Vente,741500.0,131.0,NaN,RUE,8525,SAINT DENIS,75001.0,...,NaN,75101,75001 PARIS 01,131 RUE SAINT DENIS 75001 PARIS,PARIS,12155.737705,1,Paris,Île-de-France,0


## Import du fichier adresses avec gps et iris

In [40]:
adresses = pd.read_csv('03 - export_gps_iris_ALL.csv', dtype = {'code_iris_clean': str,
                                                                                             'Code departement' : str,
                                                                                             'Code postal 5 chiffres' : str,
                                                                                             'Code commune INSEE': str})
adresses.shape

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1114244, 20)

In [41]:
adresses[-adresses['code_iris_clean'].isna()].head(40)

,Unnamed: 0,Unnamed: 0.1,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,lon,lat,Code departement,Code commune,Code commune INSEE,coordonnees_gps,Code postal 5 chiffres,code_iris_clean
0,0,0,NaN,NaN,FORTUNAT,1250.0,CEYZERIAT,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,1072.0,46.183217,5.320072,5.323007,46.173753,01,NaN,NaN,NaN,01250,10720000.0
1,1,1,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,1203.0,46.246992,4.899335,4.892457,46.246330,01,NaN,NaN,NaN,01290,12030000.0
2,2,2,NaN,NaN,BOIS DU CHAMP RION,1290.0,LAIZ,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,1203.0,46.246992,4.899335,4.892457,46.246330,01,NaN,NaN,NaN,01290,12030000.0
3,3,3,NaN,NaN,EN COROBERT,1290.0,LAIZ,1290 LAIZ,EN COROBERT 1290 LAIZ,1203.0,46.246992,4.899335,4.881913,46.248368,01,NaN,NaN,NaN,01290,12030000.0
4,4,4,NaN,RUE,DE LA CHARTREUSE,1960.0,PERONNAS,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,1289.0,46.171190,5.218022,5.227583,46.184979,01,NaN,NaN,NaN,01960,12890102.0
5,5,5,NaN,NaN,CHAMP PORTIER,1370.0,VAL-REVERMONT,1370 VAL REVERMONT,CHAMP PORTIER 1370 VAL-REVERMONT,NaN,NaN,NaN,5.342953,46.264482,01,NaN,NaN,NaN,01370,014260000
6,6,6,NaN,NaN,CHAMPEL,1270.0,COLIGNY,1270 COLIGNY,CHAMPEL 1270 COLIGNY,1108.0,46.386664,5.330823,5.349373,46.380992,01,NaN,NaN,NaN,01270,11080000.0
7,7,7,5367.0,NaN,LORETTE,1270.0,COLIGNY,1270 COLIGNY,5367 LORETTE 1270 COLIGNY,1108.0,46.386664,5.330823,5.351500,46.380469,01,NaN,NaN,NaN,01270,11080000.0
8,8,8,31.0,RUE,COMTE DE LA TEYSSONNIERE,1000.0,BOURG-EN-BRESSE,1000 BOURG EN BRESSE,31 RUE COMTE DE LA TEYSSONNIERE 1000 BOURG-EN-...,1053.0,46.205152,5.246021,5.208966,46.197026,01,NaN,NaN,NaN,01000,10530203.0
9,9,9,6.0,AV,DU CHAMP DE FOIRE,1000.0,BOURG-EN-BRESSE,1000 BOURG EN BRESSE,6 AV DU CHAMP DE FOIRE 1000 BOURG-EN-BRESSE,1053.0,46.205152,5.246021,5.229923,46.206965,01,NaN,NaN,NaN,01000,10530101.0


In [42]:
adresses['corse'] = adresses['code_iris_clean'].str[0:2]
adresses = adresses[(adresses['corse'] != '2A') & (adresses['corse'] != '2B')]
adresses = adresses.drop(columns = {'corse'})
adresses.shape

adresses['code_iris_clean'] = adresses['code_iris_clean'].fillna(-1).astype(float).astype(int).astype(str)
adresses['code_iris_clean'] = adresses['code_iris_clean'].apply(lambda x: x.zfill(9))
adresses['code_iris_clean'] = adresses['code_iris_clean'].astype(str).replace('-00000001', np.nan)

adresses.head()

,Unnamed: 0,Unnamed: 0.1,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,lon,lat,Code departement,Code commune,Code commune INSEE,coordonnees_gps,Code postal 5 chiffres,code_iris_clean
0,0,0,NaN,NaN,FORTUNAT,1250.0,CEYZERIAT,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,1072.0,46.183217,5.320072,5.323007,46.173753,01,NaN,NaN,NaN,01250,010720000
1,1,1,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,1203.0,46.246992,4.899335,4.892457,46.246330,01,NaN,NaN,NaN,01290,012030000
2,2,2,NaN,NaN,BOIS DU CHAMP RION,1290.0,LAIZ,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,1203.0,46.246992,4.899335,4.892457,46.246330,01,NaN,NaN,NaN,01290,012030000
3,3,3,NaN,NaN,EN COROBERT,1290.0,LAIZ,1290 LAIZ,EN COROBERT 1290 LAIZ,1203.0,46.246992,4.899335,4.881913,46.248368,01,NaN,NaN,NaN,01290,012030000
4,4,4,NaN,RUE,DE LA CHARTREUSE,1960.0,PERONNAS,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,1289.0,46.171190,5.218022,5.227583,46.184979,01,NaN,NaN,NaN,01960,012890102


In [43]:
# On supprime les duplicates éventuels
adresses.drop_duplicates()
adresses.shape

(1112315, 20)

In [44]:
df.columns

Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Code commune INSEE',
       'Ville', 'Adresse', 'Commune Alt', 'Prix m2', 'Paris', 'dep_name',
       'region_name', 'Mer'],
      dtype='object')

In [45]:
adresses.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'No voie', 'Type de voie', 'Voie',
       'Code postal', 'Commune', 'Ville', 'Adresse', 'INSEE_COM',
       'Commune_Lat', 'Commune_Lon', 'lon', 'lat', 'Code departement',
       'Code commune', 'Code commune INSEE', 'coordonnees_gps',
       'Code postal 5 chiffres', 'code_iris_clean'],
      dtype='object')

In [46]:
adresses = adresses.drop(columns = {'Unnamed: 0', 'Unnamed: 0.1', 'No voie', 'Type de voie', 'Voie', 'Code postal',
                                   'Commune', 'Ville', 'INSEE_COM', 'Code departement', 'Code commune', 'Code commune INSEE',
                                    'coordonnees_gps'})
adresses.head()

,Adresse,Commune_Lat,Commune_Lon,lon,lat,Code postal 5 chiffres,code_iris_clean
0,FORTUNAT 1250 CEYZERIAT,46.183217,5.320072,5.323007,46.173753,01250,010720000
1,TERRES DES CINQ SAULES 1290 LAIZ,46.246992,4.899335,4.892457,46.246330,01290,012030000
2,BOIS DU CHAMP RION 1290 LAIZ,46.246992,4.899335,4.892457,46.246330,01290,012030000
3,EN COROBERT 1290 LAIZ,46.246992,4.899335,4.881913,46.248368,01290,012030000
4,RUE DE LA CHARTREUSE 1960 PERONNAS,46.171190,5.218022,5.227583,46.184979,01960,012890102


In [47]:
df = df.merge(adresses, on = "Adresse", how = "left")
df.head()

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,Paris,dep_name,region_name,Mer,Commune_Lat,Commune_Lon,lon,lat,Code postal 5 chiffres,code_iris_clean
0,1,02/01/2020,Vente,268800.0,14.0,NaN,RUE,0411,DE L'ARCADE,75008.0,...,1,Paris,Île-de-France,0,NaN,NaN,2.323688,48.871652,75008,751083106
1,1,06/01/2020,Vente,878378.0,9.0,NaN,BD,5888,DE LA MADELEINE,75001.0,...,1,Paris,Île-de-France,0,NaN,NaN,2.326961,48.869560,75001,751010402
2,1,10/01/2020,Vente,680000.0,10.0,NaN,RUE,1200,DU BOURG L'ABBE,75003.0,...,1,Paris,Île-de-France,0,NaN,NaN,2.351889,48.864062,75003,751031204
3,1,08/01/2020,Vente,370000.0,79.0,NaN,RUE,0420,DES ARCHIVES,75003.0,...,1,Paris,Île-de-France,0,NaN,NaN,2.359887,48.863220,75003,751031003
4,1,09/01/2020,Vente,741500.0,131.0,NaN,RUE,8525,SAINT DENIS,75001.0,...,1,Paris,Île-de-France,0,NaN,NaN,2.347447,48.858758,75001,751010101


In [48]:
df.to_csv('04 - dvf_with_gps_iris.csv')